<a href="https://colab.research.google.com/github/Jyoti-Hajjargi/Monthly-Bill-Generator.ipynb/blob/main/Python_Assessment_Monthly_Bill_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from datetime import datetime
from calendar import monthrange

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    """
    Generates a bill for the given month based on the item list.

    Parameters:
    item_list (list): List of dictionaries with item details.
    target_month (str): Month in "YYYY-MM" format (e.g., "2024-11").

    Returns:
    dict: A dictionary with grouped line items and total revenue.
    """

    line_items = []
    total_revenue = 0.0

    # 1. Parse the target month and year.
    year, month = map(int, target_month.split('-'))
    _, num_days_in_month = monthrange(year, month)  # Get number of days in the month.
    target_month_start = datetime(year, month, 1)
    target_month_end = datetime(year, month, num_days_in_month)

    # 2. Iterate through each item in the item_list.
    for item in item_list:
        # 3. Convert relevant date strings to datetime objects, handling potential errors.
        try:
            start_date = datetime.strptime(item['start_date'], '%Y-%m-%d')
            stop_date = datetime.strptime(item['stop_date'], '%Y-%m-%d')
        except ValueError:
            print(f"Skipping item {item.get('idx', 'Unknown')}: Invalid date format.") #handle the error
            continue  # Skip to the next item if there's a date error.

        # 4. Check for date intersection.
        #    The item is active in the target month if either the start or stop
        #    date falls within the target month, or if the target month is entirely
        #    contained within the item's start and stop dates.
        if (start_date <= target_month_end and stop_date >= target_month_start):

            # 5. Calculate the billing period.
            billing_start = max(start_date, target_month_start)
            billing_end = min(stop_date, target_month_end)
            billing_period_str = billing_start.strftime('%Y-%m-%d') + " to " + billing_end.strftime('%Y-%m-%d')

            # 6. Calculate the number of active days.
            active_days = (billing_end - billing_start).days + 1

            # 7. Convert rate and qty to numeric types, handling potential errors.
            try:
                rate = float(item.get('rate', 0))  # Default to 0 if 'rate' is missing or invalid.
                qty = int(item.get('qty', 0))      # Default to 0 if 'qty' is missing or invalid.
            except ValueError:
                print(f"Skipping item {item.get('idx', 'Unknown')}: Invalid rate or quantity.")
                continue # Skip to next item

            # 8. Calculate the amount.
            amount = rate * qty * active_days / num_days_in_month

            # 9. Check for grouping and update or append.
            grouped = False
            for line_item in line_items:
                if (line_item['item_code'] == item['item_code'] and
                        line_item['rate'] == rate and
                        line_item['billing_period'] == billing_period_str):
                    line_item['qty'] += qty
                    line_item['amount'] += amount
                    grouped = True
                    break

            if not grouped:
                line_items.append({
                    'item_code': item['item_code'],
                    'rate': rate,
                    'qty': qty,
                    'amount': amount,
                    'billing_period': billing_period_str,
                })

            # 10. Accumulate total revenue.
            total_revenue += amount

    # 11. Return the result.
    return {
        'line_items': line_items,
        'total_revenue': round(total_revenue, 2),  # Round to 2 decimal places for currency.
    }

'''if __name__ == '__main__':
    item_list = [
        { "idx": 1, "item_code": "Executive Desk (4*2)", "sales_description": "Dedicated Executive Desk", "qty": 10, "rate": "1000", "amount": "10000", "start_date": "2023-11-01", "stop_date": "2024-10-17", },
        { "idx": 2, "item_code": "Executive Desk (4*2)", "qty": "10", "rate": "1080", "amount": "10800", "start_date": "2024-10-18", "stop_date": "2025-10-31", },
        { "idx": 3, "item_code": "Executive Desk (4*2)", "qty": 15, "rate": "1080", "amount": "16200", "start_date": "2024-11-01", "stop_date": "2025-10-31", },
        { "idx": 4,  "item_code": "Executive Desk (4*2)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-01", "stop_date": "2025-10-31", },
        { "idx": 5, "item_code": "Manager Cabin", "qty": 5, "rate": 5000, "amount": 25000, "start_date": "2024-11-01", "stop_date": "2025-10-31", },
        { "idx": 6, "item_code": "Manager Cabin", "qty": 7, "rate": "5000", "amount": 35000, "start_date": "2024-12-15", "stop_date": "2025-10-31", },
        { "idx": 7, "item_code": "Manager Cabin", "qty": 10, "rate": 4600, "amount": 46000,  "start_date": "2023-11-01", "stop_date": "2024-10-17", },
        { "idx": 8, "item_code": "Parking (2S)", "qty": 10, "rate": 1000, "amount": 10000, "start_date": "2024-11-01", "stop_date": "2025-10-31", },
        { "idx": 9, "item_code": "Parking (2S)", "qty": 10, "rate": 0, "amount": 0, "start_date": "2024-11-01", "stop_date": "2025-10-31", },
        { "idx": 10, "item_code": "Executive Desk (4*2)", "qty": "8", "rate": "1100", "amount": "8800", "start_date": "2024-11-15", "stop_date": "2025-01-31", },
        {  "idx": 11, "item_code": "Manager Cabin", "qty": "3", "rate": "5200", "amount": "15600", "start_date": "2024-10-10", "stop_date": "2024-11-10", },
        { "idx": 12, "item_code": "Conference Table", "qty": 1, "rate": "20000", "amount": "20000", "start_date": "2024-11-05", "stop_date": "2024-11-20", },
        { "idx": 13, "item_code": "Parking (2S)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-15", "stop_date": "2025-02-28", },
        { "idx": 14, "item_code": "Reception Desk", "qty": 2, "rate": "7000",  "amount": "14000", "start_date": "2024-11-01", "stop_date": "2025-03-31", },
        { "idx": 15, "item_code": "Reception Desk", "qty": 1, "rate": "7000", "amount": "7000", "start_date": "2024-11-10", "stop_date": "2024-11-25", },
         { "idx": 16, "item_code": "Breakout Area", "qty": 3, "rate": "3000", "amount": "9000", "start_date": "2024-01-01", "stop_date": "2024-01-31", }

    ]
    target_month = "2024-11"
    bill = generate_monthly_bill(item_list, target_month)
    print(bill)

    target_month = "2024-10"
    bill = generate_monthly_bill(item_list, target_month)
    print(bill)

    target_month = "2024-12"
    bill = generate_monthly_bill(item_list, target_month)
    print(bill)'''


def format_output(data):
    """
    Formats the output data into the desired structure.

    Args:
        data (dict): The output data to be formatted.

    Returns:
        dict: The formatted output data.
    """
    formatted_data = {
        "line_items": [],
        "total_revenue": 0.0
    }

    for item in data['line_items']:
        formatted_item = {
            "item_code": item['item_code'],
            "rate": item['rate'],
            "qty": item['qty'],
            "amount": round(item['amount'], 2),  # Round amount to 2 decimal places
            "billing_period": item['billing_period']
        }
        formatted_data["line_items"].append(formatted_item)
        formatted_data["total_revenue"] += item['amount']

    formatted_data["total_revenue"] = round(formatted_data["total_revenue"], 2) # Round total revenue
    return formatted_data

# ... (data_month1, data_month2, data_month3 remain the same) ...


formatted_output_month1 = format_output(data_month1)
formatted_output_month2 = format_output(data_month2)
formatted_output_month3 = format_output(data_month3)

# Printing line by line for month 1:
print("Output for 2024-11:")
import json # import json module
print(json.dumps(formatted_output_month1, indent=4))  # Use json.dumps for pretty printing

# Printing line by line for month 2:
print("\nOutput for 2024-10:")
print(json.dumps(formatted_output_month2, indent=4))

# Printing line by line for month 3:
print("\nOutput for 2024-12:")
print(json.dumps(formatted_output_month3, indent=4))


Output for 2024-11:
{
    "line_items": [
        {
            "item_code": "Executive Desk (4*2)",
            "rate": 1080.0,
            "qty": 25,
            "amount": 27000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Executive Desk (4*2)",
            "rate": 1000.0,
            "qty": 5,
            "amount": 5000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Manager Cabin",
            "rate": 5000.0,
            "qty": 5,
            "amount": 25000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Parking (2S)",
            "rate": 1000.0,
            "qty": 10,
            "amount": 10000.0,
            "billing_period": "2024-11-01 to 2024-11-30"
        },
        {
            "item_code": "Parking (2S)",
            "rate": 0.0,
            "qty": 10,
            "amount": 0.0,
            "b